In [ ]:
import numpy as np
    

In [ ]:
##define the fixed parameters
hatchability = 0.98  #if hatchability is density (egg or adult) depedent, then define it in the pre-adult-module or the adult-module functions 
x1 = 2.5  #parameter in finding the mean larval size
x2 = 1  #parameter in finding the mean larval size 
x3 = 0.009 #parameter in finding the mean larval size 
sigma_size = 0.45  #parameter in assigning larval sizes by drawing from a normal distribution
mc = 1.1 #critical size cut off of the larval stage for successful pupation (= 1.1 (JB) and 1 (FEJ))
x4 = 1.0  #parameter in finding the adult sizes
female_proportion = 0.5 #assign sex to the adutls 
x5 = 85 #parameter in finding fecundity
x6 = 2  #parameter in finding fecundity
sen_adsize = 1.7 #parameter related to sensivity of fecundity to adult size
sen_adden = 0.17 #parameter related to sensivity of fecundity to adult denisity

In [ ]:
##Pre-Adult-Module
#food = larval food amt; 1.76 (LL and LH), 2.56 (HL and HH) 

"""  
This function takes the number of eggs in gen t and larval food amount as input and returns the number of adults in gen t and their size distribution as output.

Args: 
    numegg (int): number of eggs in generation t
    food (float): larval food amount in ml

Returns:
    numadult (int): number of adults in generation t
    size_adult_arr (array): size distribution of adults in generation t
"""
def Pre_Adult_Module(numegg,food):
    numlarva = int(hatchability*numegg)
    mean_size = x1*(1-1/(x2+np.exp(-x3*numlarva+food)))
    size_larva_arr = abs(np.random.normal(mean_size, sigma_size, numlarva))
    numadult = (size_larva_arr>=mc).sum()
    size_adult_arr = x4*size_larva_arr[size_larva_arr>=mc]
    return numadult, size_adult_arr

##Demographic-Stochasticity
"""  
This function takes the number of adults in gen t, if their number is less than 8 then it is reduced to 0 by 50 % chance

Args: 
    numadult (int): number of adults in generation t before demographic stochasticity

Returns:
    numadult (int): number of adults in generation t after demographic stochasticity
"""
def Demo_Stoch(numadult):
    if numadult < 8:
        numadult = np.random.binomial(size=1, n=1, p=0.5)*numadult # numadult either remains the same or is reduced to 0
    return numadult



##Adult-Module
#adnut = #adult food nutrition quality; 1 (LL and HL), 1.29 (HH) and 1.49 (LH)
"""  
This function takes the number of adults in gen t, their size distribution and the nutrition quality of adult food as inputs and returns the number of eggs in gen t+1 as output.

Args: 
    numadult (int): number of adults in generation t
    size_adult_arr (array): size distribution of adults in generation t
    adnut (float): adult food nutrition quality

Returns:
    numegg (int): number of eggs in generation t+1
"""
def Adult_Module(numadult, size_adult_arr,adnut):
    if numadult == 0: # if the population goes extinct then restart it with 4 females of size 2*mc
        numadult = 4
        size_female_arr = 2*mc*np.ones(numadult)
    else :    
        adult_sex_arr = np.random.binomial(size=numadult, n=1, p=female_proportion) # 1 is a female and 0 a male
        size_female_arr = size_adult_arr[adult_sex_arr == 1]
    addens_ind_fec_arr = adnut*x5*np.log(x6+sen_adsize*size_female_arr)
    addens_eff = 1/(1+sen_adden*numadult)
    fecundity_arr = addens_ind_fec_arr*addens_eff
    numegg = fecundity_arr.sum()
    return numegg

##Simulation
"""  
This function takes the number of eggs in gen 1, food amount provided to the larvae in each if the successive gens, the adult food nutrition quality in each if the successive gens, number of generations for which each simulation will be run and the number of replicate simulations that will be run as inputs and returns an array of adult population sizes for each generation (each column is a replicate simulation and each row is a generation) 

Args: 
    numegg (int): number of eggs in generation 1
    food (float): larval food amount in ml
    adnut (float): adult food nutrition quality
    generations (int): number of generations for which each simulation will be run
    replicates (int): number of replicate simulations that will be run

Returns:
    numadult_matrix (array): array of adult population sizes for each generation (each column is a replicate simulation and each row is a generation)
"""
def Simulation(numegg,food,adnut,generations,replicates):
    numadult_matrix = np.zeros((generations,replicates)) #array to store the number of adults per generation
    for i in range(replicates):
        # 1st generation, we start with numegg eggs
        numadult, size_adult_arr = Pre_Adult_Module(numegg,food)
        numadult = Demo_Stoch(numadult)
        if numadult == 0:
            numadult_matrix[0,i] = 4
        else:
            numadult_matrix[0,i] = numadult
        for j in range(1,generations):
            numegg = Adult_Module(numadult,size_adult_arr,adnut)
            numadult, size_adult_arr = Pre_Adult_Module(numegg,food)
            numadult = Demo_Stoch(numadult)
            if numadult == 0:
                numadult_matrix[j,i] = 4
            else:
                numadult_matrix[j,i] = numadult
    return numadult_matrix
 

In [ ]:
## Run some test simulations
# numegg = 18 (given in sec 2.3.2); rest of the parameters as in paper; run 49 generation 8 replicates simulations for each of the 4 food regimes
numegg = 18
food = [1.76,2.56] #1.76 (LL and LH), 2.56 (HL and HH)
adnut = [1,1.29,1.49] #1 (LL and HL), 1.29 (HH) and 1.49 (LH)
generations = 49
replicates = 8
LH_matrix = Simulation(numegg,food[0],adnut[2],generations,replicates)
HL_matrix = Simulation(numegg,food[1],adnut[0],generations,replicates)
LL_matrix = Simulation(numegg,food[0],adnut[0],generations,replicates)
HH_matrix = Simulation(numegg,food[1],adnut[1],generations,replicates)






In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
#boxplot of the population sizes
all_points_array = np.column_stack((LH_matrix.flatten(),HL_matrix.flatten(),LL_matrix.flatten(),HH_matrix.flatten()))
plt.boxplot(all_points_array,meanline=True,showmeans=True)
plt.ylim(0, 250)
plt.xticks([1, 2, 3, 4], ['LH', 'HL', 'LL', 'HH'])
plt.ylabel('Population size')
myHandle = [Line2D([], [], color='orange', lw = 3), Line2D([], [], color='green', linestyle = "dashed", lw = 3)]
plt.legend(handles = myHandle, labels=['median', 'mean'], frameon = True, loc = 'best', fontsize = 12)


In [ ]:
#barplot of fluctuation index
def Fluc_Ind(tim_ser):
    T = len(tim_ser)
    Nbar = np.mean(tim_ser)
    FI = 0 #initiate
    for i in range(T-1):
        FI += abs(tim_ser[i+1]-tim_ser[i])
    FI = FI/(T*Nbar)
    return FI

def FI_numadult_matrix(numadult_matrix):
    FI_arr = np.zeros(np.shape(numadult_matrix)[1])
    for i in range(np.shape(numadult_matrix)[1]):
        FI_arr[i] = Fluc_Ind(numadult_matrix[:,i])
    FI_mean = np.mean(FI_arr)
    FI_std = np.std(FI_arr)
    return FI_mean, FI_std
        

In [ ]:
LH_FI_mean, LH_FI_sd = FI_numadult_matrix(LH_matrix)
HL_FI_mean, HL_FI_sd = FI_numadult_matrix(HL_matrix)
LL_FI_mean, LL_FI_sd = FI_numadult_matrix(LL_matrix)
HH_FI_mean, HH_FI_sd = FI_numadult_matrix(HH_matrix)

In [ ]:
# Create lists for the plot
regimes = ['LH', 'HL', 'LL', 'HH']
x_pos = np.arange(len(regimes))
mean_FI = [LH_FI_mean, HL_FI_mean, LL_FI_mean, HH_FI_mean]
error_FI = [LH_FI_sd, HL_FI_sd, LL_FI_sd, HH_FI_sd]
# Build the plot
fig, ax = plt.subplots()
ax.bar(x_pos, mean_FI, yerr=error_FI, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Fluctuation Index (FI)')
ax.set_xticks(x_pos)
ax.set_xticklabels(regimes)
ax.yaxis.grid(True)
plt.yticks(np.arange(0, 2, 0.5))
plt.show()

Theoretical study of the effect of life history related parameters (hatchability,mc,sen_adden,,sen_adsize) on FI


In [ ]:
generations = 100
replicates = 100
## hatchability
hatchability_arr = np.arange(0.18,1.02,0.06)
FI_hatchability_arr = np.zeros([8,len(hatchability_arr)])
for i in range(len(hatchability_arr)):
    hatchability = hatchability_arr[i]
    LH_matrix = Simulation(numegg,food[0],adnut[2],generations,replicates)
    HL_matrix = Simulation(numegg,food[1],adnut[0],generations,replicates)
    LL_matrix = Simulation(numegg,food[0],adnut[0],generations,replicates)
    HH_matrix = Simulation(numegg,food[1],adnut[1],generations,replicates)


    LH_FI_mean, LH_FI_sd = FI_numadult_matrix(LH_matrix)
    HL_FI_mean, HL_FI_sd = FI_numadult_matrix(HL_matrix)
    LL_FI_mean, LL_FI_sd = FI_numadult_matrix(LL_matrix)
    HH_FI_mean, HH_FI_sd = FI_numadult_matrix(HH_matrix)


    FI_hatchability_arr[0,i] = LH_FI_mean
    FI_hatchability_arr[1,i] = LH_FI_sd
    FI_hatchability_arr[2,i] = HL_FI_mean
    FI_hatchability_arr[3,i] = HL_FI_sd
    FI_hatchability_arr[4,i] = LL_FI_mean
    FI_hatchability_arr[5,i] = LL_FI_sd
    FI_hatchability_arr[6,i] = HH_FI_mean
    FI_hatchability_arr[7,i] = HH_FI_sd


regimes = ['LH', 'HL', 'LL', 'HH']
markers = ['^','o','p','*']
plt.figure()
for i in range(len(regimes)):
    plt.errorbar(hatchability_arr, FI_hatchability_arr[2*i,:],FI_hatchability_arr[2*i+1,:],linestyle='solid', marker=f'{markers[i]}')
plt.legend(regimes)    


    

In [ ]:
generations = 100
replicates = 100
## MC
MC_arr = np.arange(0.9,1.4,0.05)   
FI_MC_arr = np.zeros([8,len(MC_arr)])
for i in range(len(MC_arr)):
    mc = MC_arr[i]
    LH_matrix = Simulation(numegg,food[0],adnut[2],generations,replicates)
    HL_matrix = Simulation(numegg,food[1],adnut[0],generations,replicates)
    LL_matrix = Simulation(numegg,food[0],adnut[0],generations,replicates)
    HH_matrix = Simulation(numegg,food[1],adnut[1],generations,replicates)


    LH_FI_mean, LH_FI_sd = FI_numadult_matrix(LH_matrix)
    HL_FI_mean, HL_FI_sd = FI_numadult_matrix(HL_matrix)
    LL_FI_mean, LL_FI_sd = FI_numadult_matrix(LL_matrix)
    HH_FI_mean, HH_FI_sd = FI_numadult_matrix(HH_matrix)


    FI_MC_arr[0,i] = LH_FI_mean
    FI_MC_arr[1,i] = LH_FI_sd
    FI_MC_arr[2,i] = HL_FI_mean
    FI_MC_arr[3,i] = HL_FI_sd
    FI_MC_arr[4,i] = LL_FI_mean
    FI_MC_arr[5,i] = LL_FI_sd
    FI_MC_arr[6,i] = HH_FI_mean
    FI_MC_arr[7,i] = HH_FI_sd


regimes = ['LH', 'HL', 'LL', 'HH']
markers = ['^','o','p','*']
plt.figure()
for i in range(len(regimes)):
    plt.errorbar(MC_arr, FI_MC_arr[2*i,:],FI_MC_arr[2*i+1,:],linestyle='solid', marker=f'{markers[i]}')
plt.legend(regimes)    

In [ ]:
generations = 100
replicates = 100
## sen_adden
sen_adden_arr = np.arange(0.12,0.22,0.01)
FI_sen_adden_arr = np.zeros([8,len(sen_adden_arr)])
for i in range(len(sen_adden_arr)):
    sen_adden = sen_adden_arr[i]
    LH_matrix = Simulation(numegg,food[0],adnut[2],generations,replicates)
    HL_matrix = Simulation(numegg,food[1],adnut[0],generations,replicates)
    LL_matrix = Simulation(numegg,food[0],adnut[0],generations,replicates)
    HH_matrix = Simulation(numegg,food[1],adnut[1],generations,replicates)


    LH_FI_mean, LH_FI_sd = FI_numadult_matrix(LH_matrix)
    HL_FI_mean, HL_FI_sd = FI_numadult_matrix(HL_matrix)
    LL_FI_mean, LL_FI_sd = FI_numadult_matrix(LL_matrix)
    HH_FI_mean, HH_FI_sd = FI_numadult_matrix(HH_matrix)


    FI_sen_adden_arr[0,i] = LH_FI_mean
    FI_sen_adden_arr[1,i] = LH_FI_sd
    FI_sen_adden_arr[2,i] = HL_FI_mean
    FI_sen_adden_arr[3,i] = HL_FI_sd
    FI_sen_adden_arr[4,i] = LL_FI_mean
    FI_sen_adden_arr[5,i] = LL_FI_sd
    FI_sen_adden_arr[6,i] = HH_FI_mean
    FI_sen_adden_arr[7,i] = HH_FI_sd


regimes = ['LH', 'HL', 'LL', 'HH']
markers = ['^','o','p','*']
plt.figure()
for i in range(len(regimes)):
    plt.errorbar(sen_adden_arr, FI_sen_adden_arr[2*i,:],FI_sen_adden_arr[2*i+1,:],linestyle='solid', marker=f'{markers[i]}')
plt.legend(regimes)    


    

In [ ]:
generations = 100
replicates = 100
## sen_adden
sen_adden_arr = np.arange(0.12,0.22,0.01)
FI_sen_adden_arr = np.zeros([8,len(sen_adden_arr)])
for i in range(len(sen_adden_arr)):
    sen_adden = sen_adden_arr[i]
    LH_matrix = Simulation(numegg,food[0],adnut[2],generations,replicates)
    HL_matrix = Simulation(numegg,food[1],adnut[0],generations,replicates)
    LL_matrix = Simulation(numegg,food[0],adnut[0],generations,replicates)
    HH_matrix = Simulation(numegg,food[1],adnut[1],generations,replicates)


    LH_FI_mean, LH_FI_sd = FI_numadult_matrix(LH_matrix)
    HL_FI_mean, HL_FI_sd = FI_numadult_matrix(HL_matrix)
    LL_FI_mean, LL_FI_sd = FI_numadult_matrix(LL_matrix)
    HH_FI_mean, HH_FI_sd = FI_numadult_matrix(HH_matrix)


    FI_sen_adden_arr[0,i] = LH_FI_mean
    FI_sen_adden_arr[1,i] = LH_FI_sd
    FI_sen_adden_arr[2,i] = HL_FI_mean
    FI_sen_adden_arr[3,i] = HL_FI_sd
    FI_sen_adden_arr[4,i] = LL_FI_mean
    FI_sen_adden_arr[5,i] = LL_FI_sd
    FI_sen_adden_arr[6,i] = HH_FI_mean
    FI_sen_adden_arr[7,i] = HH_FI_sd


regimes = ['LH', 'HL', 'LL', 'HH']
markers = ['^','o','p','*']
plt.figure()
for i in range(len(regimes)):
    plt.errorbar(sen_adden_arr, FI_sen_adden_arr[2*i,:],FI_sen_adden_arr[2*i+1,:],linestyle='solid', marker=f'{markers[i]}')
plt.legend(regimes)    

In [ ]:
generations = 100
replicates = 100
## sen_adsize
sen_adsize_arr = np.arange(1.2,2.2,0.1)
FI_sen_adsize_arr = np.zeros([8,len(sen_adsize_arr)])
for i in range(len(sen_adsize_arr)):
    sen_adsize = sen_adsize_arr[i]
    LH_matrix = Simulation(numegg,food[0],adnut[2],generations,replicates)
    HL_matrix = Simulation(numegg,food[1],adnut[0],generations,replicates)
    LL_matrix = Simulation(numegg,food[0],adnut[0],generations,replicates)
    HH_matrix = Simulation(numegg,food[1],adnut[1],generations,replicates)


    LH_FI_mean, LH_FI_sd = FI_numadult_matrix(LH_matrix)
    HL_FI_mean, HL_FI_sd = FI_numadult_matrix(HL_matrix)
    LL_FI_mean, LL_FI_sd = FI_numadult_matrix(LL_matrix)
    HH_FI_mean, HH_FI_sd = FI_numadult_matrix(HH_matrix)


    FI_sen_adsize_arr[0,i] = LH_FI_mean
    FI_sen_adsize_arr[1,i] = LH_FI_sd
    FI_sen_adsize_arr[2,i] = HL_FI_mean
    FI_sen_adsize_arr[3,i] = HL_FI_sd
    FI_sen_adsize_arr[4,i] = LL_FI_mean
    FI_sen_adsize_arr[5,i] = LL_FI_sd
    FI_sen_adsize_arr[6,i] = HH_FI_mean
    FI_sen_adsize_arr[7,i] = HH_FI_sd


regimes = ['LH', 'HL', 'LL', 'HH']
markers = ['^','o','p','*']
plt.figure()
for i in range(len(regimes)):
    plt.errorbar(sen_adsize_arr, FI_sen_adsize_arr[2*i,:],FI_sen_adsize_arr[2*i+1,:],linestyle='solid', marker=f'{markers[i]}')
plt.legend(regimes)    


    